# Bag of Words
### Probs : Sentence similarity & machine learning model's input data
### Cons  : Sparsity, Frequent words has more power, Ignoring word orders, Out of Vocabulary

# n-gram : next word prediction, miss spell checker

# TF-IDF : TF score, IDF : log(total # of Docs/ # of Docs with the term in it+ 1)
log(분자에 문장(문서수) / 단어의 빈도수) 

Euclidean Distance Similarity

Cosine Similarity 

In [2]:
# Drawback : 단어 있고 없다만으로 있다보니, 동음이의어, 또는 비슷한 문장을 완전다르게 인식, 문장 유사도 인식의문제가 있음, 

# LSA 잠재의미분석 : 토픽기반

단어와 문장을 행과 열로 매트릭스화 해서 SVD 로 표현 : 단어 X 강도 X 문장 Cosine similarity 측정

# Word Embeddings (Word2Vec, Glove)

In [4]:
# 텍스트 > 숫자인코딩 > 원핫 인코딩 > 유사도를 위해 (거리, 코사인), > 임베딩(저차원, 유사도를 갖음)

In [25]:
### Word2Vec data generation(skipgram, window size=(2))
"king brave man"
"queen beautyful woman"

'queen beautyful woman'

In [26]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

# Remove stop words

In [27]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [28]:
corpus = remove_stop_words(corpus)

In [29]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)

In [30]:
words

{'boy',
 'girl',
 'king',
 'man',
 'pretty',
 'prince',
 'princess',
 'queen',
 'strong',
 'wise',
 'woman',
 'young'}

In [31]:
word2int = {}
for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [32]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

king strong man
queen wise woman
boy young man
girl young woman
prince young king
princess young queen
man strong
woman pretty
prince boy king
princess girl queen


In [33]:
df.head(10)

,input,label
0,king,strong
1,king,man
2,strong,king
3,strong,man
4,man,king
5,man,strong
6,queen,wise
7,queen,woman
8,wise,queen
9,wise,woman


In [34]:
df.shape

(52, 2)

In [35]:
word2int

{'princess': 0,
 'queen': 1,
 'pretty': 2,
 'girl': 3,
 'prince': 4,
 'strong': 5,
 'wise': 6,
 'king': 7,
 'boy': 8,
 'woman': 9,
 'man': 10,
 'young': 11}

# Define Tensorflow Graph¶

In [40]:
import tensorflow.compat.v1 as tf
import tensorflow as tf
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [24]:
!tf_upgrade_v2 -h

usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--no_import_rename] [--no_upgrade_compat_v1_import]
                     [--reportfile REPORT_FILENAME] [--mode {DEFAULT,SAFETY}]
                     [--print_all]

Convert a TensorFlow Python file from 1.x to 2.0

Simple usage:
  tf_upgrade_v2.py --infile foo.py --outfile bar.py
  tf_upgrade_v2.py --infile foo.ipynb --outfile bar.ipynb
  tf_upgrade_v2.py --intree ~/code/old --outtree ~/code/new

optional arguments:
  -h, --help            show this help message and exit
  --infile INPUT_FILE   If converting a single file, the name of the file to
                        convert
  --outfile OUTPUT_FILE
                        If converting a single file, the output filename.
  --intree INPUT_TREE   If converting a whole tree of files, the directory to
        

In [37]:
print(tf.__version__)

2.3.0-dev20200611


# ConceptNet